In [ ]:
from pysnmp.entity.rfc3413.oneliner import cmdgen

cmd = cmdgen.CommandGenerator()

In [ ]:
errorIndication, errorStatus, errorIndex, varBinds = cmd.getCmd(
    # v1
    # cmdgen.CommunityData('public', 0),
    # v2
    # cmdgen.CommunityData('public'),
    # v3
    cmdgen.UsmUserData(
        'demo', 'my_new_password', 'my_new_password', 
        cmdgen.usmHMACMD5AuthProtocol,
        cmdgen.usmDESPrivProtocol
    ),
    cmdgen.UdpTransportTarget(('ss.laisky.com', 161)),
    cmdgen.MibVariable('1.1.1.1.1.1.1.1.0')
)

print(
    'errorIndication: {}\n'
    'errorStatus: {}\n'
    'errorIndex: {}\n'
    'varBinds: {}'
    .format(errorIndication, errorStatus, errorIndex, varBinds)
)

chexiang
===

- system = 1.1.1.1.2.1.1
- sysName = 1.1.1.1.1.1.1.1
- ifEntry = 1.1.1.1.1.1.1.1.1
- ifDescr = 1.1.1.1.1.1.1.1.1.2
- ifSpeed = 1.1.1.1.1.1.1.1.1.5

## 同步

In [ ]:
import string
from pysnmp.smi import builder, view, error
from pysnmp.entity.rfc3413.oneliner import cmdgen


# 创建 cmd generator
cmdGen = cmdgen.CommandGenerator()  # 同步

mibBuilder = builder.MibBuilder()
mibBuilder.loadModules(
    'RFC-1213', 'RFC-3413'
)
mibView = view.MibViewController(mibBuilder)



errorIndication, errorStatus, errorIndex, varBinds = cmdGen.nextCmd(
    cmdgen.CommunityData('huawei@123'),
    cmdgen.UdpTransportTarget(('1.1.1.1', 161)),
    cmdgen.MibVariable('IF-MIB', 'ifOperStatus')
)


print(errorIndication)
print(errorStatus)
print(errorIndex)
print('*' * 50)

for row in varBinds:
    for oid, val in row:
        print(oid.getMibSymbol()[-1][0], val.prettyPrint())

## 异步

In [3]:
import string
import time
# from pysnmp.smi import builder, view, error
from pysnmp.entity.rfc3413.oneliner import cmdgen


# 创建 cmd generator
# cmdGen = cmdgen.CommandGenerator()  # 同步
cmdGen = cmdgen.AsynCommandGenerator()  # 异步

# mibBuilder = builder.MibBuilder()
# mibBuilder.loadModules(
#     'RFC-1213', 'RFC-3413'
# )
# mibView = view.MibViewController(mibBuilder)


mib_vars = (
#     {'desc': 'ifOperStatus', 'mib': cmdgen.MibVariable('IF-MIB', 'ifOperStatus')},  # 端口状态
#     {'desc': 'ifSpeed', 'mib': cmdgen.MibVariable('IF-MIB', 'ifSpeed')},  # 端口速度 bits/s
#     {'desc': 'ifDescr', 'mib': cmdgen.MibVariable('IF-MIB', 'ifDescr')},
#     {'desc': 'ifInOctets', 'mib': cmdgen.MibVariable('IF-MIB', 'ifInOctets')},
#     {'desc': 'ifOutOctets', 'mib': cmdgen.MibVariable('IF-MIB', 'ifOutOctets')},
    {'desc': 'multi', 'mibs': (cmdgen.MibVariable('IF-MIB', 'ifDescr'), 
                               cmdgen.MibVariable('IF-MIB', 'ifSpeed'),
                               cmdgen.MibVariable('IF-MIB', 'ifInOctets'),
                              )},

)


def callback(sendRequestHandle, errorIndication, errorStatus, errorIndex, varBinds, cbCtx):
    if errorIndication:
        print(errorIndication)
        print(errorStatus)
        print(errorIndex)
        return 1
        
    for row in varBinds:
        for oid, val in row:
            c = 0
            mib = oid.getLabel()[-1]
            if mib not in ('ifDescr', 'ifSpeed', 'ifInOctets'):
                return
            
            print(oid.getLabel()[-1], str(int(oid.getMibSymbol()[-1][0])), val.prettyPrint())
            
    return 1  # 告诉 walk 继续运行
            


for mib_var in mib_vars:
    cbCtx = {'object_name': mib_var['desc']}
    cmdGen.nextCmd(
        cmdgen.CommunityData('huawei@123'),
        cmdgen.UdpTransportTarget(('1.1.1.1', 161), timeout=1, retries=3),
#         0, 25,
#         (mib_var['mib'],),
        mib_var['mibs'],
        (callback, cbCtx),
        True, True
    )

    
# 启动
print('start job')
# cmdGen.snmpEngine.transportDispatcher.jobStarted(1)
print('run dispatcher')
cmdGen.snmpEngine.transportDispatcher.runDispatcher()
print('close dispatcher')
# cmdGen.snmpEngine.transportDispatcher.closeDispatcher()
print('job finished')
# cmdGen.snmpEngine.transportDispatcher.
# cmdGen.snmpEngine.transportDispatcher.jobFinished(1)


start job
run dispatcher
ifDescr 1 NULL0
ifSpeed 1 0
ifInOctets 1 0
ifDescr 2 InLoopBack0
ifSpeed 2 0
ifInOctets 2 0
ifDescr 3 MEth0/0/0
ifSpeed 3 10000000
ifInOctets 3 0
ifDescr 5 GE1/0/1
ifSpeed 5 100000000
ifInOctets 5 21486960
ifDescr 6 GE1/0/2
ifSpeed 6 100000000
ifInOctets 6 73831400
ifDescr 7 GE1/0/3
ifSpeed 7 100000000
ifInOctets 7 20616779
ifDescr 8 GE1/0/4
ifSpeed 8 100000000
ifInOctets 8 137803186
ifDescr 9 GE1/0/5
ifSpeed 9 100000000
ifInOctets 9 14925197
ifDescr 10 GE1/0/6
ifSpeed 10 100000000
ifInOctets 10 254609481
ifDescr 11 GE1/0/7
ifSpeed 11 100000000
ifInOctets 11 242136131
ifDescr 12 GE1/0/8
ifSpeed 12 100000000
ifInOctets 12 242045815
ifDescr 13 GE1/0/9
ifSpeed 13 1000000000
ifInOctets 13 0
ifDescr 14 GE1/0/10
ifSpeed 14 1000000000
ifInOctets 14 0
ifDescr 15 GE1/0/11
ifSpeed 15 1000000000
ifInOctets 15 2088424162
ifDescr 16 GE1/0/12
ifSpeed 16 1000000000
ifInOctets 16 1422586693
ifDescr 17 GE1/0/13
ifSpeed 17 1000000000
ifInOctets 17 915814353
ifDescr 18 GE1/0/14
i